# How controls will work

- engine begins with initial fuel setting and resistor level

### Control States

- lightoff sequencing
  - normal process with initial fuel setting

- manual mode
  - same as lightoff
  - manual fues settings adjusted

- PID
  - initialize PID with last fuel setting as initial
  - implement ramp to avoid large changes
  - adjust PID from parameter page
  - adjust n2 setpoint from control page


## Command logic

- PID commands
- Stage commands
- Speed setpoint
- Fuel AO
- Drivetek
- Safeties
- Air/CO2
  

## Power electronics testing

- Testing
- ramp load
- drivetek speed commands
- what if EMI/dt errors

## Passive

- fuel output setting
- pid sequence


In [8]:
# PID control udpates

# library imports
from simple_pid import PID  # pid loop for n2

# get from redis...
p_p, p_i, p_d = 5, .1, 0
init_ao = 55

# initialize at start of program
n2_pid = PID(p_p, p_i, p_d, setpoint=20)
n2_pid.sample_time = 0.5
n2_pid.output_limits = (0, 100)

# ending or initializing always do!
n2_pid.set_auto_mode(False)

# PID begins
n2_pid.set_auto_mode(True, last_output=init_ao)

# calling pid loop with max change
def call_pid(val, prev_ao):
    prev_o = n2_pid
    new_ao = n2_pid(val)
    delta = new_ao-prev_ao

    # prevent new value from being more than 3
    if new_ao > prev_ao:
        new_ao = min(prev_ao+3, new_ao)
    else:
        new_ao = max(prev_ao-3, new_ao)

    return new_ao

n2_val = 25
fuel_ao = call_pid(n2_val, init_ao)
print(fuel_ao)


52


In [ ]:
# listening for changes from control_ui. Put in state machine logic
pub_msg = 'msg'

# PID updates and control mode

if 'cv_pid_p' in pub_msg:
    # set pid values
    kp, ki, kd = pub_msg['cv_pid_p'], pub_msg['cv_pid_i'], pub_msg['cv_pid_d']
    n2_pid.tunings = (kp, ki, kd)
    
    # set redis values in hash map
    pass

elif 'cv_control_mode' in pub_msg:
    cm = pub_msg['cv_control_mode']

elif 'cv_n2' in pub_msg:
    n2_sp = pub_msg['cv_n2']
    n2_pid.setpoint = n2_sp
    # set on redis

# fuel setting
elif 'cv_fuel_ao' in pub_msg:
    # set on redis
    pass

# fuel setting
elif 'cv_ll' in pub_msg:
    # set on redis
    pass


In [22]:
# n2 simulator speed
def speed_response(fuel_percent, load_level):
    n2_max_sim = 60
    # equation came up with for simulating... not representative
    n2_speed = n2_max_sim * (fuel_percent/100) * ((10/load_level)/2)
    n1_speed = n2_speed * 3
    return round(n1_speed, 1), round(n2_speed, 1)

speed_response(100, 6)

(150.0, 50.0)